In [3]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import json

In [4]:
# 1. Load Sarvam-1 (Base Model - 2B)
model_id = "sarvamai/sarvam-1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float16,
    device_map="cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

In [5]:
# 2. Few-Shot Templates (Crucial for Base Models)
TEMPLATES = {
    "hi": "संदर्भ: {context}\nप्रश्न: {question}\nउत्तर:",
    "ta": "சூழல்: {context}\nகேள்வி: {question}\nபதில்:",
    "mr": "संदर्भ: {context}\nप्रश्न: {question}\nउत्तर:",
    "kn": "ಸನ್ನಿವೇಶ: {context}\nಪ್ರಶ್ನೆ: {question}\nಉತ್ತರ:",
    "en": "Context: {context}\nQuestion: {question}\nAnswer:"
    }

In [7]:
# def generate_sarvam(context, question, lang="hi"):
#     prompt = TEMPLATES.get(lang, TEMPLATES["en"]).format(context=context, question=question)
#     inputs = tokenizer(prompt, return_tensors="pt").to("mps")

#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=15, # Keep short for Factoid QA
#             min_length=1,
#             do_sample=False,   # Greedy decoding = Reproducible
#             pad_token_id=tokenizer.eos_token_id
#         )

#     # Cut off prompt, keep only new text
#     generated_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
#     return generated_text.split('\n')[0].strip() # Stop at newline

In [11]:
def generate_sarvam_quest(question):

    prompt = (
        "You are a great language model.\n"
        "You will answer the following questions in the manner shown\n"
        "Question: What is the capital of India?\nAnswer: New Delhi\n"
        "Now answer the following\n"
        f"Question: {question}\nAnswer:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300, # Keep short for Factoid QA
            min_length=1,
            do_sample=False,   # Greedy decoding = Reproducible
            pad_token_id=tokenizer.eos_token_id
        )

    # Cut off prompt, keep only new text
    generated_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return generated_text.split('\n')[0].strip() # Stop at newline


In [12]:
data_indic_quest = pd.read_csv("/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/English.csv")
print(f"data_indic_quest columns: {data_indic_quest.columns}")
print(data_indic_quest.head())

data_indic_quest columns: Index(['Question', 'Answer', 'Domain'], dtype='object')
                                            Question  \
0  Whose death coincided with the launch of the n...   
1  Which Indian activist was popularly known as ‘...   
2  Who was the first president of india and the f...   
3             Who was Abdul Kalam? describe briefly.   
4  Who was the finance minister of India during M...   

                                              Answer    Domain  
0  On 1 August 1920, Non-Cooperation Movement was...  Politics  
1  Rao Bahadur Gopal Hari Deshmukh () also known ...  Politics  
2  Rajendra Prasad was an Indian politician, lawy...  Politics  
3  A.P.J. Abdul Kalam (born October 15, 1931, Ram...  Politics  
4  Arun Jaitley (28 December 1952 – 24 August 201...  Politics  


In [13]:
print("🚀 Running Sarvam Generation...")
data_indic_quest['model_response'] = data_indic_quest.apply(lambda x: generate_sarvam_quest(x['Question']), axis=1)
data_indic_quest.to_csv("/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_en.csv", index=False)
print("✅ Done.")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚀 Running Sarvam Generation...


KeyboardInterrupt: 

In [14]:
generate_sarvam_quest("What is the financial capital of India?")

'Mumbai'

In [17]:
import torch
import pandas as pd
from tqdm import tqdm
import math

# --- 1. CONFIGURATION ---
BATCH_SIZE = 32  # Sarvam-1 (2B) is tiny. T4 can handle 32 or 64 easily.
OUTPUT_PATH = "/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_en_partial.csv"

# --- 2. CRITICAL: SET PADDING SIDE ---
# For generation, padding must be on the LEFT so it doesn't mess up the output
tokenizer.padding_side = "left"
# Set pad token if missing (common in base models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 3. BATCH GENERATION FUNCTION ---
def generate_batch(questions):
    # A. Apply Prompt Template to the whole batch
    prompts = []
    for q in questions:
        prompt = (
            "You are a great language model.\n"
            "Question: What is the capital of India?\nAnswer: New Delhi\n"
            f"Question: {q}\nAnswer:"
        )
        prompts.append(prompt)

    # B. Tokenize the whole batch (Pad to longest in batch)
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # C. Generate for the whole batch at once
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            min_length=1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # D. Decode and Clean
    # Calculate where the new tokens start (input length)
    input_len = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_len:]
    decoded_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Clean up newlines
    return [ans.split('\n')[0].strip() for ans in decoded_batch]

# --- 4. THE LOOP (Real-time Progress) ---
print(f"🚀 Processing {len(data_indic_quest)} rows in batches of {BATCH_SIZE}...")

results = []
data_indic_quest_hi = pd.read_csv("/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/English.csv")
questions = data_indic_quest['Question'].tolist()
total_batches = math.ceil(len(questions) / BATCH_SIZE)

# TQDM gives you the progress bar
for i in tqdm(range(0, len(questions), BATCH_SIZE), total=total_batches, desc="Generating"):
    # Get batch of questions
    batch_q = questions[i : i + BATCH_SIZE]

    # Run Inference
    batch_answers = generate_batch(batch_q)
    results.extend(batch_answers)

# --- 5. FINAL SAVE ---
data_indic_quest['model_response'] = results
final_path = "/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_en.csv"
data_indic_quest.to_csv(final_path, index=False)
print(f"✅ Done! Saved to {final_path}")

🚀 Processing 200 rows in batches of 32...


Generating: 100%|██████████| 7/7 [00:10<00:00,  1.55s/it]

✅ Done! Saved to /content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_en.csv


In [23]:
import torch
import pandas as pd
from tqdm import tqdm
import math

# --- 1. CONFIGURATION ---
BATCH_SIZE = 32  # Sarvam-1 (2B) is tiny. T4 can handle 32 or 64 easily.


# --- 2. CRITICAL: SET PADDING SIDE ---
# For generation, padding must be on the LEFT so it doesn't mess up the output
tokenizer.padding_side = "left"
# Set pad token if missing (common in base models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 3. BATCH GENERATION FUNCTION ---
def generate_batch(questions):
    # A. Apply Prompt Template to the whole batch
    prompts = []
    for q in questions:
        prompt = (
            "आप एक बेहतरीन भाषा मॉडल हैं।\n"
            "प्रश्न: भारत की राजधानी क्या है?\nउत्तर: नई दिल्ली\n"
            f"प्रश्न: {q}\nउत्तर:"
        )
        prompts.append(prompt)

    # B. Tokenize the whole batch (Pad to longest in batch)
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # C. Generate for the whole batch at once
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            min_length=1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # D. Decode and Clean
    # Calculate where the new tokens start (input length)
    input_len = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_len:]
    decoded_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Clean up newlines
    return [ans.split('\n')[0].strip() for ans in decoded_batch]

# --- 4. THE LOOP (Real-time Progress) ---
print(f"🚀 Processing {len(data_indic_quest)} rows in batches of {BATCH_SIZE}...")

results = []

data_indic_quest_hi = pd.read_csv("/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Hindi.csv")
print(data_indic_quest_hi.head())
questions_hi = data_indic_quest_hi['Question'].tolist()
total_batches = math.ceil(len(questions_hi) / BATCH_SIZE)

# TQDM gives you the progress bar
for i in tqdm(range(0, len(questions_hi), BATCH_SIZE), total=total_batches, desc="Generating"):
    # Get batch of questions
    batch_q = questions_hi[i : i + BATCH_SIZE]

    # Run Inference
    batch_answers = generate_batch(batch_q)
    results.extend(batch_answers)

# --- 5. FINAL SAVE ---
data_indic_quest_hi['model_response'] = results
final_path = "/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_hi.csv"
data_indic_quest_hi.to_csv(final_path, index=False)
print(f"✅ Done! Saved to {final_path}")

🚀 Processing 200 rows in batches of 32...
                                            Question  \
0   1920 में असहयोग आंदोलन की शुरुआत के साथ ही कि...   
1   कौन से भारतीय कार्यकर्ता लोकप्रिय रूप से 'लोक...   
2   भारत के प्रथम राष्ट्रपति और संयुक्त राज्य अमे...   
3        अब्दुल कलाम कौन थे? संक्षेप में वर्णन करें।   
4   मोदी के पहले कार्यकाल के दौरान भारत के वित्त ...   

                                              Answer    Domain  
0   1 अगस्त 1920 को असहयोग आंदोलन की घोषणा हुई, उ...  Politics  
1   राव बहादुर गोपाल हरि देशमुख () जिन्हें लोकहित...  Politics  
2   राजेंद्र प्रसाद एक भारतीय राजनीतिज्ञ, वकील, प...  Politics  
3   एपीजे अब्दुल कलाम (जन्म 15 अक्टूबर, 1931, राम...  Politics  
4   अरुण जेटली (28 दिसंबर 1952 - 24 अगस्त 2019) ए...  Politics  


Generating: 100%|██████████| 7/7 [00:09<00:00,  1.40s/it]

✅ Done! Saved to /content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_hi.csv


In [24]:
import torch
import pandas as pd
from tqdm import tqdm
import math

# --- 1. CONFIGURATION ---
BATCH_SIZE = 32  # Sarvam-1 (2B) is tiny. T4 can handle 32 or 64 easily.


# --- 2. CRITICAL: SET PADDING SIDE ---
# For generation, padding must be on the LEFT so it doesn't mess up the output
tokenizer.padding_side = "left"
# Set pad token if missing (common in base models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 3. BATCH GENERATION FUNCTION ---
def generate_batch(questions):
    # A. Apply Prompt Template to the whole batch
    prompts = []
    for q in questions:
        prompt = (
            "ನೀವು ಉತ್ತಮ ಭಾಷಾ ಮಾದರಿ.\n"
            "ಪ್ರಶ್ನೆ: ಭಾರತದ ರಾಜಧಾನಿ ಯಾವುದು?\nಉತ್ತರ: ನವದೆಹಲಿ\n"
            f"ಪ್ರಶ್ನೆ: {q}\nಉತ್ತರ:"
        )
        prompts.append(prompt)

    # B. Tokenize the whole batch (Pad to longest in batch)
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # C. Generate for the whole batch at once
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            min_length=1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # D. Decode and Clean
    # Calculate where the new tokens start (input length)
    input_len = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_len:]
    decoded_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Clean up newlines
    return [ans.split('\n')[0].strip() for ans in decoded_batch]

# --- 4. THE LOOP (Real-time Progress) ---
print(f"🚀 Processing {len(data_indic_quest)} rows in batches of {BATCH_SIZE}...")

results = []

data_indic_quest= pd.read_csv("/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Kannada.csv")
print(data_indic_quest.head())
questions = data_indic_quest['Question'].tolist()
total_batches = math.ceil(len(questions) / BATCH_SIZE)

# TQDM gives you the progress bar
for i in tqdm(range(0, len(questions), BATCH_SIZE), total=total_batches, desc="Generating"):
    # Get batch of questions
    batch_q = questions[i : i + BATCH_SIZE]

    # Run Inference
    batch_answers = generate_batch(batch_q)
    results.extend(batch_answers)

# --- 5. FINAL SAVE ---
data_indic_quest['model_response'] = results
final_path = "/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_ka.csv"
data_indic_quest.to_csv(final_path, index=False)
print(f"✅ Done! Saved to {final_path}")

🚀 Processing 200 rows in batches of 32...
                                            Question  \
0   1920 ರಲ್ಲಿ ಅಸಹಕಾರ ಚಳುವಳಿಯ ಪ್ರಾರಂಭದೊಂದಿಗೆ ಯಾರ ...   
1   ಯಾವ ಭಾರತೀಯ ಕಾರ್ಯಕರ್ತನನ್ನು 'ಲೋಖಿತ್ವಾಡಿ' ಎಂದು ಜ...   
2   ಭಾರತದ ಮೊದಲ ಅಧ್ಯಕ್ಷರು ಮತ್ತು USA ನ ಮೊದಲ ಅಧ್ಯಕ್ಷ...   
3          ಅಬ್ದುಲ್ ಕಲಾಂ ಯಾರು? ಸಂಕ್ಷಿಪ್ತವಾಗಿ ವಿವರಿಸಿ.   
4   ಮೋದಿಯವರ ಮೊದಲ ಅವಧಿಯಲ್ಲಿ ಭಾರತದ ಹಣಕಾಸು ಮಂತ್ರಿ ಯಾರು?   

                                              Answer    Domain  
0   1 ಆಗಸ್ಟ್ 1920 ರಂದು, ಅಸಹಕಾರ ಚಳುವಳಿಯನ್ನು ಘೋಷಿಸಲ...  Politics  
1   ರಾವ್ ಬಹದ್ದೂರ್ ಗೋಪಾಲ್ ಹರಿ ದೇಶಮುಖ್ () ಅವರು ಲೋಖಿ...  Politics  
2   ರಾಜೇಂದ್ರ ಪ್ರಸಾದ್ ಅವರು ಭಾರತೀಯ ರಾಜಕಾರಣಿ, ವಕೀಲರು...  Politics  
3   ಎಪಿಜೆ ಅಬ್ದುಲ್ ಕಲಾಂ (ಜನನ ಅಕ್ಟೋಬರ್ 15, 1931, ರಾ...  Politics  
4   ಅರುಣ್ ಜೇಟ್ಲಿ (28 ಡಿಸೆಂಬರ್ 1952 - 24 ಆಗಸ್ಟ್ 20...  Politics  


Generating: 100%|██████████| 7/7 [00:10<00:00,  1.52s/it]

✅ Done! Saved to /content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_ka.csv


In [25]:
import torch
import pandas as pd
from tqdm import tqdm
import math

# --- 1. CONFIGURATION ---
BATCH_SIZE = 32  # Sarvam-1 (2B) is tiny. T4 can handle 32 or 64 easily.


# --- 2. CRITICAL: SET PADDING SIDE ---
# For generation, padding must be on the LEFT so it doesn't mess up the output
tokenizer.padding_side = "left"
# Set pad token if missing (common in base models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 3. BATCH GENERATION FUNCTION ---
def generate_batch(questions):
    # A. Apply Prompt Template to the whole batch
    prompts = []
    for q in questions:
        prompt = (
            "तुम्ही एक उत्तम भाषेचे मॉडेल आहात.\n"
            "प्रश्न: भारताची राजधानी कोणती आहे?\nउत्तर: नवी दिल्ली\n"
            f"प्रश्न: {q}\nउत्तर:"
        )
        prompts.append(prompt)

    # B. Tokenize the whole batch (Pad to longest in batch)
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # C. Generate for the whole batch at once
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            min_length=1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # D. Decode and Clean
    # Calculate where the new tokens start (input length)
    input_len = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_len:]
    decoded_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Clean up newlines
    return [ans.split('\n')[0].strip() for ans in decoded_batch]

# --- 4. THE LOOP (Real-time Progress) ---
print(f"🚀 Processing {len(data_indic_quest)} rows in batches of {BATCH_SIZE}...")

results = []

data_indic_quest= pd.read_csv("/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Marathi.csv")
print(data_indic_quest.head())
questions = data_indic_quest['Question'].tolist()
total_batches = math.ceil(len(questions) / BATCH_SIZE)

# TQDM gives you the progress bar
for i in tqdm(range(0, len(questions), BATCH_SIZE), total=total_batches, desc="Generating"):
    # Get batch of questions
    batch_q = questions[i : i + BATCH_SIZE]

    # Run Inference
    batch_answers = generate_batch(batch_q)
    results.extend(batch_answers)

# --- 5. FINAL SAVE ---
data_indic_quest['model_response'] = results
final_path = "/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_ma.csv"
data_indic_quest.to_csv(final_path, index=False)
print(f"✅ Done! Saved to {final_path}")

🚀 Processing 200 rows in batches of 32...
                                            Question  \
0   1920 मध्ये असहकार चळवळ सुरू झाली तेव्हा कोणाच...   
1   कोणता भारतीय कार्यकर्ता 'लोकहितवादी' या नावान...   
2   भारताचे पहिले राष्ट्रपती आणि अमेरिकेचे पहिले ...   
3          अब्दुल कलाम कोण होते? थोडक्यात वर्णन करा.   
4   मोदींच्या पहिल्या कार्यकाळात भारताचे अर्थमंत्...   

                                              Answer    Domain  
0   1 ऑगस्ट 1920 रोजी असहकार आंदोलनाची घोषणा झाली...  Politics  
1   राव बहादूर गोपाळ हरी देशमुख () हे लोकहितवादी ...  Politics  
2   राजेंद्र प्रसाद हे एक भारतीय राजकारणी, वकील, ...  Politics  
3   APJ अब्दुल कलाम (जन्म: 15 ऑक्टोबर 1931, रामेश...  Politics  
4   अरुण जेटली (28 डिसेंबर 1952 - 24 ऑगस्ट 2019) ...  Politics  


Generating: 100%|██████████| 7/7 [00:11<00:00,  1.68s/it]

✅ Done! Saved to /content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_ma.csv


In [26]:
import torch
import pandas as pd
from tqdm import tqdm
import math

# --- 1. CONFIGURATION ---
BATCH_SIZE = 32  # Sarvam-1 (2B) is tiny. T4 can handle 32 or 64 easily.


# --- 2. CRITICAL: SET PADDING SIDE ---
# For generation, padding must be on the LEFT so it doesn't mess up the output
tokenizer.padding_side = "left"
# Set pad token if missing (common in base models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 3. BATCH GENERATION FUNCTION ---
def generate_batch(questions):
    # A. Apply Prompt Template to the whole batch
    prompts = []
    for q in questions:
        prompt = (
            "நீங்க ஒரு சிறந்த மொழி மாதிரி.\n"
            "கேள்வி: இந்தியாவின் தலைநகரம் என்ன?\nபதில்: புது தில்லி\n"
            f"கேள்வி: {q}\nபதில்:"
        )
        prompts.append(prompt)

    # B. Tokenize the whole batch (Pad to longest in batch)
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # C. Generate for the whole batch at once
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            min_length=1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # D. Decode and Clean
    # Calculate where the new tokens start (input length)
    input_len = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_len:]
    decoded_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Clean up newlines
    return [ans.split('\n')[0].strip() for ans in decoded_batch]

# --- 4. THE LOOP (Real-time Progress) ---
print(f"🚀 Processing {len(data_indic_quest)} rows in batches of {BATCH_SIZE}...")

results = []

data_indic_quest= pd.read_csv("/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Tamil.csv")
print(data_indic_quest.head())
questions = data_indic_quest['Question'].tolist()
total_batches = math.ceil(len(questions) / BATCH_SIZE)

# TQDM gives you the progress bar
for i in tqdm(range(0, len(questions), BATCH_SIZE), total=total_batches, desc="Generating"):
    # Get batch of questions
    batch_q = questions[i : i + BATCH_SIZE]

    # Run Inference
    batch_answers = generate_batch(batch_q)
    results.extend(batch_answers)

# --- 5. FINAL SAVE ---
data_indic_quest['model_response'] = results
final_path = "/content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_ta.csv"
data_indic_quest.to_csv(final_path, index=False)
print(f"✅ Done! Saved to {final_path}")

🚀 Processing 200 rows in batches of 32...
                                            Question  \
0  1920ல் ஒத்துழையாமை இயக்கம் தொடங்கப்பட்டபோது யா...   
1  லோகித்வாடி' என்று பிரபலமாக அறியப்பட்ட இந்திய ஆ...   
2  இந்தியாவின் முதல் ஜனாதிபதி மற்றும் அமெரிக்காவி...   
3           அப்துல் கலாம் யார்? சுருக்கமாக விவரிக்க.   
4  மோடியின் முதல் ஆட்சிக் காலத்தில் இந்தியாவின் ந...   

                                              Answer    Domain  
0  1 ஆகஸ்ட் 1920 அன்று ஒத்துழையாமை இயக்கம் அறிவிக...  Politics  
1  லோகித்வாடி என்றும் அழைக்கப்படும் ராவ் பகதூர் க...  Politics  
2  ராஜேந்திர பிரசாத் ஒரு இந்திய அரசியல்வாதி, வழக்...  Politics  
3  APJ அப்துல் கலாம் (பிறப்பு: அக்டோபர் 15, 1931,...  Politics  
4  அருண் ஜெட்லி (28 டிசம்பர் 1952 - 24 ஆகஸ்ட் 201...  Politics  


Generating: 100%|██████████| 7/7 [00:11<00:00,  1.59s/it]

✅ Done! Saved to /content/drive/MyDrive/685-Adv NLP/Project/Dataset/IndicQuest/Results/sarvam_results_ta.csv
